# Analyzing Profitability of Apps in the Apple App Store and Google Play Store 

### This project is meant to simulate a request I received from a developer/PM. Our company is trying to attract users towards our app and would like my understanding on what is currently popular. Our app is free and geared towards English users, so we want to make sure our popoulation has similar attributes. 

## Exploring Data

We have two datasets from both the App Store and Google Play Store. Below, I open and explore both files. Datasets are from Kaggle found [here](https://link-url-here.org) and [here](https://www.kaggle.com/datasets/lava18/google-play-store-apps). 

In [1]:
#Opening the App Store file 

open_apple_file = open("AppleStore.csv", encoding="utf8")
#Importing reader function 
from csv import reader 
read_apple = reader(open_apple_file)
ios_data = list(read_apple)
ios_data = ios_data[1:]
#Opening the Google Play Store file 

open_google_file = open("googleplaystore.csv", encoding="utf8")
#Importing reader function 
read_google = reader(open_google_file)
google_data = list(read_google)
google_data = google_data[1:]

In [2]:
#exploring the App Store data

#using a function that will help slice and format data 

def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [3]:
explore_data(ios_data, 0, 5, True)
explore_data(google_data, 0, 5, True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


Number of rows: 7197
Number of columns: 16
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '

I have opened and explored the data. Now, I want to clean the data to ensure there aren't any errors or inconsistencies.

## Cleaning Data

In [4]:
#Kaggle discussion stated that the Google Play dataset has an error for one of its row
print(google_data[10472])
print(len(google_data))
# We are going to remove that row 

del google_data[10472] 
print(len(google_data))

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
10841
10840


In [5]:
# Checking if it got deleted 
print(google_data[10472])

['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']


We are going to check for any duplicate rows in either the App Store or Google Play store. 

In [6]:
#Making empty lists
duplicate_ios_apps = []
ios_unique_apps = [] 
for app in ios_data:
    name = app[0]
    if name in ios_unique_apps: 
        duplicate_ios_apps.append(name)
    else: 
        ios_unique_apps.append(name)

print('Number of duplicate apps:', len(duplicate_ios_apps))
print('\n')
print('Examples of duplicate apps:', duplicate_ios_apps[:15])

Number of duplicate apps: 0


Examples of duplicate apps: []


In [7]:
duplicate_google_apps = []
google_unique_apps = [] 
for app in google_data:
    name = app[0]
    if name in google_unique_apps: 
        duplicate_google_apps.append(name)
    else: 
        google_unique_apps.append(name)

print('Number of duplicate apps:', len(duplicate_google_apps))
print('\n')
print('Examples of duplicate apps:', duplicate_google_apps[:15])
#Seeing there are duplicates, I want to see what the expected length should be 
print('Expected length:', len(google_data) - 1181)

Number of duplicate apps: 1181


Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']
Expected length: 9659


The App Store data does not have duplicates but the Google Play Store data has 1181 duplicates. Therefore, we will remove these duplicates to clean up this dataset. However, not just any duplicate will be removed. I will keep entries with the highest number of reviews (making the assumption these entries are most recent) and remove the rest.  

To remove duplicates, I created a new dictionary. This dictionary serves as my reference, because I only want a unique entry of each app along with the highest number of ratings associated with it. Therefore, if the app name is not already in reviews_max, then it will be added to the dictionary. However, if there is already an entry for the name in the dictionary, we will keep the app that has the highest number of ratings. 

In [8]:
reviews_max = {}
for app in google_data: 
    name = app[0]
    n_reviews = float(app[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews 
    elif name not in reviews_max: 
        reviews_max[name] = n_reviews 

#Making sure we have the correct expected length before we move forward         
print(len(reviews_max))

9659


Now that we have a dictionary for each app and its highest number of ratings, we are going to create a list that contains all of our clean data for the Google Play Store in android_clean. For bookkeeping purposes, we will use the empty list already_added to make sure we only add each app once. If the app name is not in already_added and the entry for the app has the highest number of ratings (which we can match with the dictionary reviews_max we made earlier) then the entire row from google_data will be added to android_clean. 

In [9]:
android_clean = []
already_added = []
for app in google_data: 
    name = app[0]
    n_reviews = float(app[3])
    if (name not in already_added) and (reviews_max[name] == n_reviews):
        android_clean.append(app)
        already_added.append(name)

In [10]:
explore_data(android_clean, 0, 5, True)
#As we can see we have 9659 entries which is what we want  

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


Number of rows: 9659
Number of columns: 13


## Analyzing Data

We want to ensure that all of the apps we are looking at are designed for English-speaking audiences since that is what type of app we have. However, based on the documentation from Kaggle, we know that there are a few foreign apps we have to remove. Letters associated with the English alphabet can be parsed using the ord() function and are numbered 0 to 127. Therefore, if a number is equal to or less than 127, the character belongs to a set of common English characters. However, some English app names can have weird characters, so only apps have more than three characters greater than 127 will be removed. 

In [11]:
def is_english(a_string):
    non_english = 0 
    for character in a_string:
        if ord(character) > 127: 
            non_english += 1 
        
    if non_english > 3: 
        return False
    else: 
        return True

In [12]:
#Finding Google Play Store English apps 
english_android_apps = []
for app in android_clean: 
    name = app[0]
    if is_english(name):
        english_android_apps.append(app)

In [13]:
#Finding App Store Store English apps 
english_ios_apps = []
for app in ios_data: 
    name = app[1]
    if is_english(name):
        english_ios_apps.append(app)

In [14]:
print(len(english_android_apps))
print(len(english_ios_apps))

9614
6183


Now, we want to make sure all the apps we analyze are also free. 

In [15]:
#Isolating free apps in App Store 
final_ios = []
for app in english_ios_apps: 
    price = app[4]
    if price == '0.0': 
        final_ios.append(app)

In [16]:
#Isolating free apps in Google Play Store 
final_google = []
for app in english_android_apps: 
    price = app[7]
    if price == '0': 
        final_google.append(app)

In [17]:
print(len(final_google))
print(len(final_ios))

8864
3222


We want to make that we can launch a profitable app in both the App Store and the Google Play Store to maximize revenue. We also want to ensure that we can launch the app without too many costs. 

Now that there are two clean datasets, I will analyze what is the most popular genre in both stores. I will use the prime_genre column for the App Store and the Genres and Category columns for the Google Play Set. 

In [18]:
def freq_table(dataset, index):
    new_dictionary = {}
    for row in dataset:
        x = row[index]
        if x in new_dictionary:
            new_dictionary[x] += 1 
        else: 
            new_dictionary[x] = 1
    
    percentages_table = {}
    length = len(dataset)
    for x in new_dictionary: 
        proportion = new_dictionary[x]/length 
        percentage = proportion*100 
        percentages_table[x] = percentage
    return percentages_table

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [19]:
display_table(final_ios, 11) #Prime Genre 

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


Above, I found the percentage of the most popular apps by genre in the App Store. It looks like Games are an overwhelming majority with 58.2% with Entertainment in second at 7.9%. Games take up most the App Store categories for free and English speaking apps. It looks like in general, most apps are designed for entertainment.  

In [20]:
display_table(final_google, 1) #Category 

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

In [21]:
display_table(final_google, 9) #Genres

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

Evaluating the data from the Google Play Store, the top genres for both Category and Genre columns are Family and Tools with 18.9% and 8.4% respectively. The apps for the Play Store look to be more practical in comparison to the App Store where a majority of apps were designed for entertainment purposes. Therefore, based on this analysis, I will do more analysis to provide a better determination on what type of app would be the most profitable. 

Next, I will evaluate the datasets to see which apps have the most users. To do so, I will use the Installs column for the Google Play Store and the rating_count_tot column for the App Store. 


In [22]:
unique_genres = freq_table(final_ios, 11) #Prime Genre 

In [23]:
for genre in unique_genres:
    total = 0 
    len_genre = 0
    
    for row in final_ios:
        genre_app = row[11]
        if genre_app == genre:
            user_rating_counts = float(row[5])
            total += user_rating_counts 
            len_genre += 1 
    
    avg_user_ratings = total/len_genre 
    
    print(genre, '-', round(avg_user_ratings,2))

Social Networking - 71548.35
Photo & Video - 28441.54
Games - 22788.67
Music - 57326.53
Reference - 74942.11
Health & Fitness - 23298.02
Weather - 52279.89
Utilities - 18684.46
Travel - 28243.8
Shopping - 26919.69
News - 21248.02
Navigation - 86090.33
Lifestyle - 16485.76
Entertainment - 14029.83
Food & Drink - 33333.92
Sports - 23008.9
Book - 39758.5
Finance - 31467.94
Education - 7003.98
Productivity - 21028.41
Business - 7491.12
Catalogs - 4004.0
Medical - 612.0


Evaluating popular apps, the Navigation genre has the highest user rating counts in the App Store with ~86k ratings, followed closely by Reference with ~75k ratings. Let's take a closer look at both these categories. 

In [24]:
for app in final_ios:
    genre = app[11]
    if genre == 'Navigation' and app[5] != 0: 
        print('Navigation: ', app[1], app[5], '\n')

Navigation:  Waze - GPS Navigation, Maps & Real-time Traffic 345046 

Navigation:  Google Maps - Navigation & Transit 154911 

Navigation:  Geocaching® 12811 

Navigation:  CoPilot GPS – Car Navigation & Offline Maps 3582 

Navigation:  ImmobilienScout24: Real Estate Search in Germany 187 

Navigation:  Railway Route Search 5 



In [25]:
for app in final_ios:
    genre = app[11]
    if genre == 'Reference' and app[5] != 0: 
        print('Reference: ', app[1], app[5], '\n')

Reference:  Bible 985920 

Reference:  Dictionary.com Dictionary & Thesaurus 200047 

Reference:  Dictionary.com Dictionary & Thesaurus for iPad 54175 

Reference:  Google Translate 26786 

Reference:  Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran 18418 

Reference:  New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition 17588 

Reference:  Merriam-Webster Dictionary 16849 

Reference:  Night Sky 12122 

Reference:  City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) 8535 

Reference:  LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools 4693 

Reference:  GUNS MODS for Minecraft PC Edition - Mods Tools 1497 

Reference:  Guides for Pokémon GO - Pokemon GO News and Cheats 826 

Reference:  WWDC 762 

Reference:  Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free 718 

Reference:  VPN Express 14 

Reference:  Real Bike Traffic Rider Virtual Reality Glasses 

The Navigation App has fewer apps but a few apps such as Waze and Google Maps skewing its average while Reference has more apps with higher ratings, but due to the volume of Social Networking apps, the average of ratings per genre is lower. It might be beneficial to create an app in the Reference space since users are more likely to intereact with them and leave a rating. 

Next, let's evaluate the Google Play Store. While the Google Play Store has more information about the number of installations, they are bucketed into categories such as 10k+, 100k+, etc. which doesn't give us a finite number. So I took the values as is, for example, if an app has 10k+ installs, then I took it as if they had 10k installs to be able to computate values. 

In [26]:
unique_category = freq_table(final_google, 1) #Category 

In [27]:
for category in unique_category:
    total = 0 
    len_category = 0
    
    for row in final_google:
        category_app = row[1]
        if category_app == category:
            installs = row[5]
            installs = installs.replace('+', '')
            installs = installs.replace(',', '')
            install_counts = float(installs)
            total += install_counts 
            len_category += 1 
    
    avg_user_ratings = total/len_category 
    
    print(category, '-', round(avg_user_ratings,2))

ART_AND_DESIGN - 1986335.09
AUTO_AND_VEHICLES - 647317.82
BEAUTY - 513151.89
BOOKS_AND_REFERENCE - 8767811.89
BUSINESS - 1712290.15
COMICS - 817657.27
COMMUNICATION - 38456119.17
DATING - 854028.83
EDUCATION - 1833495.15
ENTERTAINMENT - 11640705.88
EVENTS - 253542.22
FINANCE - 1387692.48
FOOD_AND_DRINK - 1924897.74
HEALTH_AND_FITNESS - 4188821.99
HOUSE_AND_HOME - 1331540.56
LIBRARIES_AND_DEMO - 638503.73
LIFESTYLE - 1437816.27
GAME - 15588015.6
FAMILY - 3695641.82
MEDICAL - 120550.62
SOCIAL - 23253652.13
SHOPPING - 7036877.31
PHOTOGRAPHY - 17840110.4
SPORTS - 3638640.14
TRAVEL_AND_LOCAL - 13984077.71
TOOLS - 10801391.3
PERSONALIZATION - 5201482.61
PRODUCTIVITY - 16787331.34
PARENTING - 542603.62
WEATHER - 5074486.2
VIDEO_PLAYERS - 24727872.45
NEWS_AND_MAGAZINES - 9549178.47
MAPS_AND_NAVIGATION - 4056941.77


Evaluating popular apps, the Communication genre has the most installations with ~38M installations, followed by Video_Players and Social, with ~24M and ~23M respectively. Let's dig into these categories further. 

In [28]:
for app in final_google:
    genre = app[1]
    if genre == 'COMMUNICATION' and app[5] != 0: 
        print('COMMUNICATION: ', app[0], app[5], '\n')

COMMUNICATION:  WhatsApp Messenger 1,000,000,000+ 

COMMUNICATION:  Messenger for SMS 10,000,000+ 

COMMUNICATION:  My Tele2 5,000,000+ 

COMMUNICATION:  imo beta free calls and text 100,000,000+ 

COMMUNICATION:  Contacts 50,000,000+ 

COMMUNICATION:  Call Free – Free Call 5,000,000+ 

COMMUNICATION:  Web Browser & Explorer 5,000,000+ 

COMMUNICATION:  Browser 4G 10,000,000+ 

COMMUNICATION:  MegaFon Dashboard 10,000,000+ 

COMMUNICATION:  ZenUI Dialer & Contacts 10,000,000+ 

COMMUNICATION:  Cricket Visual Voicemail 10,000,000+ 

COMMUNICATION:  TracFone My Account 1,000,000+ 

COMMUNICATION:  Xperia Link™ 10,000,000+ 

COMMUNICATION:  TouchPal Keyboard - Fun Emoji & Android Keyboard 10,000,000+ 

COMMUNICATION:  Skype Lite - Free Video Call & Chat 5,000,000+ 

COMMUNICATION:  My magenta 1,000,000+ 

COMMUNICATION:  Android Messages 100,000,000+ 

COMMUNICATION:  Google Duo - High Quality Video Calls 500,000,000+ 

COMMUNICATION:  Seznam.cz 1,000,000+ 

COMMUNICATION:  Antillean Gold

In [29]:
for app in final_google:
    genre = app[1]
    if genre == 'VIDEO_PLAYERS' and app[5] != 0: 
        print('VIDEO_PLAYERS: ', app[0], app[5], '\n')

VIDEO_PLAYERS:  YouTube 1,000,000,000+ 

VIDEO_PLAYERS:  All Video Downloader 2018 1,000,000+ 

VIDEO_PLAYERS:  Video Downloader 10,000,000+ 

VIDEO_PLAYERS:  HD Video Player 1,000,000+ 

VIDEO_PLAYERS:  Iqiyi (for tablet) 1,000,000+ 

VIDEO_PLAYERS:  Video Player All Format 10,000,000+ 

VIDEO_PLAYERS:  Motorola Gallery 100,000,000+ 

VIDEO_PLAYERS:  Free TV series 100,000+ 

VIDEO_PLAYERS:  Video Player All Format for Android 500,000+ 

VIDEO_PLAYERS:  VLC for Android 100,000,000+ 

VIDEO_PLAYERS:  Code 10,000,000+ 

VIDEO_PLAYERS:  Vote for 50,000,000+ 

VIDEO_PLAYERS:  XX HD Video downloader-Free Video Downloader 1,000,000+ 

VIDEO_PLAYERS:  OBJECTIVE 1,000,000+ 

VIDEO_PLAYERS:  Music - Mp3 Player 10,000,000+ 

VIDEO_PLAYERS:  HD Movie Video Player 1,000,000+ 

VIDEO_PLAYERS:  YouCut - Video Editor & Video Maker, No Watermark 5,000,000+ 

VIDEO_PLAYERS:  Video Editor,Crop Video,Movie Video,Music,Effects 1,000,000+ 

VIDEO_PLAYERS:  YouTube Studio 10,000,000+ 

VIDEO_PLAYERS:  vide

In [30]:
for app in final_google:
    genre = app[1]
    if genre == 'SOCIAL' and app[5] != 0: 
        print('SOCIAL: ', app[0], app[5], '\n')

SOCIAL:  Facebook 1,000,000,000+ 

SOCIAL:  Facebook Lite 500,000,000+ 

SOCIAL:  Tumblr 100,000,000+ 

SOCIAL:  Social network all in one 2018 100,000+ 

SOCIAL:  Pinterest 100,000,000+ 

SOCIAL:  TextNow - free text + calls 10,000,000+ 

SOCIAL:  Google+ 1,000,000,000+ 

SOCIAL:  The Messenger App 1,000,000+ 

SOCIAL:  Messenger Pro 1,000,000+ 

SOCIAL:  Free Messages, Video, Chat,Text for Messenger Plus 1,000,000+ 

SOCIAL:  Telegram X 5,000,000+ 

SOCIAL:  The Video Messenger App 100,000+ 

SOCIAL:  Jodel - The Hyperlocal App 1,000,000+ 

SOCIAL:  Hide Something - Photo, Video 5,000,000+ 

SOCIAL:  Love Sticker 1,000,000+ 

SOCIAL:  Web Browser & Fast Explorer 5,000,000+ 

SOCIAL:  LiveMe - Video chat, new friends, and make money 10,000,000+ 

SOCIAL:  VidStatus app - Status Videos & Status Downloader 5,000,000+ 

SOCIAL:  Love Images 1,000,000+ 

SOCIAL:  Web Browser ( Fast & Secure Web Explorer) 500,000+ 

SOCIAL:  SPARK - Live random video chat & meet new people 5,000,000+ 

SOC

Similar to what we saw with the App Store, we have a few apps in the Google Play Store that skew the data due to their mass downloads. These apps such as What's App in the Communication market dominate the market. So it may be useful to go with our previous recommendation to build an app in the Reference category which doesn't have much competition but has a higher volume of users.  

### Thank you ;)

That's all folks! This was a fun project and I hope you had a fun time going through this insane text. 